In [68]:
import pandas as pd
import dill as pickle
import requests
import json
import warnings
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import Pipeline
from sklearn.base import TransformerMixin, BaseEstimator
import numpy as np

pd.set_option('display.max_columns', 50)

### Конвейер обучения и применения модели через Pipeline

Подгружаем данные [отсюда](https://www.kaggle.com/datasets/abcsds/pokemon) и изучаем их 

In [69]:
data = pd.read_csv('Pokemon.csv')
data.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


In [70]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   #           800 non-null    int64 
 1   Name        800 non-null    object
 2   Type 1      800 non-null    object
 3   Type 2      414 non-null    object
 4   Total       800 non-null    int64 
 5   HP          800 non-null    int64 
 6   Attack      800 non-null    int64 
 7   Defense     800 non-null    int64 
 8   Sp. Atk     800 non-null    int64 
 9   Sp. Def     800 non-null    int64 
 10  Speed       800 non-null    int64 
 11  Generation  800 non-null    int64 
 12  Legendary   800 non-null    bool  
dtypes: bool(1), int64(9), object(3)
memory usage: 75.9+ KB


In [71]:
# Если я хочу получить номера индексов, на местах которых стоит 'False'?

data['Legendary'].value_counts()

Legendary
False    735
True      65
Name: count, dtype: int64

Будем прогнозировать значение переменной Legendary по остальным признакам, приведем его в числовому виду.  
Поле # и имя нам не понадобятся.  
Также удаляем поле Total, поскольку оно выражается через другие поля.  

In [72]:
data.drop(['#', 'Name', 'Total'], axis=True, inplace=True)
data.head()

,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,Grass,Poison,45,49,49,65,65,45,1,False
1,Grass,Poison,60,62,63,80,80,60,1,False
2,Grass,Poison,80,82,83,100,100,80,1,False
3,Grass,Poison,80,100,123,122,120,80,1,False
4,Fire,NaN,39,52,43,60,50,65,1,False


Отделяем признаки от целевой переменной.

In [73]:
features = data.drop(['Legendary'], axis=1)
target = 1 * data['Legendary']
target

0      0
1      0
2      0
3      0
4      0
      ..
795    1
796    1
797    1
798    1
799    1
Name: Legendary, Length: 800, dtype: int64

Классы сбалансированны?

In [74]:
target.value_counts()

Legendary
0    735
1     65
Name: count, dtype: int64

Отделим обучающую часть от тестовой

In [75]:
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.3, random_state=43)

Посмотрим на значения категориальных переменных

In [76]:
features_train['Type 1'].unique()

array(['Water', 'Normal', 'Fire', 'Dark', 'Grass', 'Bug', 'Fighting',
       'Ground', 'Dragon', 'Ice', 'Poison', 'Fairy', 'Steel', 'Ghost',
       'Psychic', 'Electric', 'Flying', 'Rock'], dtype=object)

In [77]:
features_train['Type 2'].unique()

array([nan, 'Flying', 'Normal', 'Dragon', 'Psychic', 'Poison', 'Ground',
       'Dark', 'Fighting', 'Water', 'Electric', 'Fire', 'Ice', 'Ghost',
       'Steel', 'Fairy', 'Rock', 'Grass', 'Bug'], dtype=object)

In [78]:
types_all = sorted(set(pd.concat([features_train['Type 1'], features_train['Type 2']]).dropna()))
types_all
data.columns

Index(['Type 1', 'Type 2', 'HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def',
       'Speed', 'Generation', 'Legendary'],
      dtype='object')

Развернем типы во флаговые колонки: для каждого типа будет колонка, в которой значение 1 будет, если такой тип у покемона есть, а иначе - значение 0

In [79]:
class PokemonTypesToFlags(TransformerMixin, BaseEstimator):

    def __init__(self):
        pass

    def fit(self, X, y=None):
        self.types_all = list(set(pd.concat([X['Type 1'], X['Type 2']]).dropna()))
        return self

    def transform(self, X):
        X = X.reset_index(drop=True)
        
        ohe_types = pd.DataFrame.from_dict(
            list(
                X
                .apply(
                    lambda f: dict([
                        (
                            t, 1 if f['Type 1'] == t or f['Type 2'] == t else 0
                        )
                        for t in self.types_all
                    ]),
                    axis=1
                )
            )
        )
        
        X = pd.concat([X.drop(['Type 1', 'Type 2'], axis=1), ohe_types], axis=1)
        return X

Преобразуем тренировочный датасет 

In [80]:
pttf = PokemonTypesToFlags()
pttf.fit(features_train)
features_train_pptf = pttf.transform(features_train)
features_train_pptf.head()

,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Flying,Normal,Electric,Ground,Dark,Rock,Ice,Grass,Fairy,Fighting,Poison,Psychic,Bug,Steel,Fire,Ghost,Water,Dragon
0,80,80,80,80,80,80,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,40,55,30,30,30,60,4,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,79,103,120,135,115,78,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,62,50,58,73,54,72,6,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4,90,120,100,150,120,100,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1


Отмасштабируем признаки

In [81]:
scaler = StandardScaler()
scaler.fit(features_train_pptf)
features_train_pptf_scaled = pd.DataFrame(scaler.transform(features_train_pptf), columns=scaler.feature_names_in_)
features_train_pptf_scaled.head()

,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Flying,Normal,Electric,Ground,Dark,Rock,Ice,Grass,Fairy,Fighting,Poison,Psychic,Bug,Steel,Fire,Ghost,Water,Dragon
0,0.456275,0.039980,0.199806,0.223015,0.338148,0.404960,0.424085,-0.377964,-0.390209,-0.258199,-0.306186,-0.237915,-0.258199,-0.211604,-0.377964,-0.237915,-0.269809,-0.302685,-0.349635,-0.330017,-0.254242,-0.309662,-0.242077,2.300464,-0.233696
1,-1.179014,-0.741748,-1.387753,-1.323398,-1.588623,-0.281932,0.424085,2.645751,2.562727,-0.258199,-0.306186,-0.237915,-0.258199,-0.211604,-0.377964,-0.237915,-0.269809,-0.302685,-0.349635,-0.330017,-0.254242,-0.309662,-0.242077,-0.434695,-0.233696
2,0.415392,0.759170,1.469853,1.924069,1.686888,0.336270,-1.361535,-0.377964,-0.390209,-0.258199,-0.306186,-0.237915,-0.258199,-0.211604,-0.377964,-0.237915,-0.269809,-0.302685,-0.349635,-0.330017,-0.254242,-0.309662,-0.242077,2.300464,-0.233696
3,-0.279605,-0.898094,-0.498720,0.006517,-0.663773,0.130203,1.614497,-0.377964,2.562727,-0.258199,-0.306186,-0.237915,-0.258199,-0.211604,-0.377964,-0.237915,-0.269809,-0.302685,-0.349635,-0.330017,-0.254242,3.229330,-0.242077,-0.434695,-0.233696
4,0.865097,1.290745,0.834829,2.387992,1.879566,1.091852,0.424085,-0.377964,-0.390209,-0.258199,-0.306186,-0.237915,-0.258199,-0.211604,-0.377964,-0.237915,-0.269809,-0.302685,-0.349635,-0.330017,-0.254242,-0.309662,-0.242077,2.300464,4.279059


Построим простой классификатор с помощью полученных признаков и оценим его качество

In [82]:
model = DecisionTreeClassifier(random_state=0, class_weight='balanced')
model.fit(features_train_pptf_scaled, target_train)
# print('Качество модели на обучающей выборке:', {roc_auc_score(target_train, model.predict_proba(features_train_pptf_scaled)[:, 1])})
print('Качество модели на обучающей выборке:', {roc_auc_score(target_train, model.predict(features_train_pptf_scaled))})    

Качество модели на обучающей выборке: {1.0}


Подготовим конвейнер, чтобы можно было запустить все эти трансформеры последовательно одной командой

In [83]:
pipe = Pipeline([  
    ('ohe_types', PokemonTypesToFlags()),
    ('scaler', StandardScaler()),
    ('classify', DecisionTreeClassifier(class_weight='balanced', random_state=0))
])

Настраиваем трансформеры однократно на исходной обучающей выборке и проверяем, что они работают, как ожидается.

In [84]:
pipe.fit(X=features_train, y=target_train)
# print('Качество модели на обучающей выборке:', {roc_auc_score(target_train, pipe.predict_proba(features_train)[:, 1])})
print('Качество модели на обучающей выборке:', {roc_auc_score(target_train, pipe.predict(features_train))})

Качество модели на обучающей выборке: {1.0}


Проверим качество на тестовой части

In [85]:
print('Качество модели на тестовой выборке (roc_auc):', {roc_auc_score(target_test, pipe.predict_proba(features_test)[:, 1])})

Качество модели на тестовой выборке (roc_auc): {0.7051532941943901}


Модель явно переобучилась. А какая у дерева глубина?

In [86]:
pipe['classify'].tree_.max_depth

20

Переберем несколько вариантов, с помощью кросс-валидации, чтобы улучшить качество на тестовой выборке

In [87]:
pipe = Pipeline([  
  ('ohe_types', PokemonTypesToFlags()),
  ('scaler', StandardScaler()),
  ('classify', DecisionTreeClassifier(class_weight='balanced', random_state=0))
])

params = [
    {'classify': [LogisticRegression(class_weight='balanced', random_state=0)]}, 
    {'classify': [DecisionTreeClassifier(class_weight='balanced', random_state=0)], 'classify__max_depth': [2, 5, 10, 20]}
]

grid_search = GridSearchCV(pipe, param_grid=params, cv=5, scoring='roc_auc')
grid_search.fit(X=features_train, y=target_train)
print(
    'Качество модели на тестовой выборке c лучшей моделью:', 
    {roc_auc_score(target_test, grid_search.predict_proba(features_test)[:, 1])}
)

Качество модели на тестовой выборке c лучшей моделью: {0.989780387040661}


In [88]:
grid_search.best_estimator_

Pipeline(steps=[('ohe_types', PokemonTypesToFlags()),
                ('scaler', StandardScaler()),
                ('classify',
                 LogisticRegression(class_weight='balanced', random_state=0))])

In [89]:
pd.DataFrame.from_dict(grid_search.cv_results_).transpose()

,0,1,2,3,4
mean_fit_time,0.195697,0.253244,0.247019,0.208518,0.206336
std_fit_time,0.020389,0.06171,0.042205,0.018054,0.023568
mean_score_time,0.070648,0.072342,0.072261,0.060719,0.059213
std_score_time,0.011811,0.032546,0.017452,0.005035,0.011741
param_classify,"LogisticRegression(class_weight='balanced', ra...",DecisionTreeClassifier(class_weight='balanced'...,DecisionTreeClassifier(class_weight='balanced'...,DecisionTreeClassifier(class_weight='balanced'...,DecisionTreeClassifier(class_weight='balanced'...
param_classify__max_depth,NaN,2.0,5.0,10.0,20.0
params,{'classify': LogisticRegression(class_weight='...,{'classify': DecisionTreeClassifier(class_weig...,{'classify': DecisionTreeClassifier(class_weig...,{'classify': DecisionTreeClassifier(class_weig...,{'classify': DecisionTreeClassifier(class_weig...
split0_test_score,0.985577,0.874399,0.891226,0.895433,0.778846
split1_test_score,0.952535,0.796117,0.690399,0.713592,0.661812
split2_test_score,0.985976,0.772384,0.816613,0.756203,0.75836


In [90]:
grid_search.best_estimator_

Pipeline(steps=[('ohe_types', PokemonTypesToFlags()),
                ('scaler', StandardScaler()),
                ('classify',
                 LogisticRegression(class_weight='balanced', random_state=0))])

### Оборачивание модели в сервис API

Сериализуем (консервируем) модель

In [91]:
with open('best_pokemon_model.pk', 'wb') as file:
    pickle.dump(grid_search, file)

Десериализуем ее, чтобы убедиться, что она правильно работает

In [92]:
with open('best_pokemon_model.pk','rb') as f:
    loaded_model = pickle.load(f)

In [93]:
print(
    'Качество модели на тестовой выборке от законсервированной модели:', 
    {roc_auc_score(target_test, loaded_model.predict_proba(features_test)[:, 1])}
)

Качество модели на тестовой выборке от законсервированной модели: {0.989780387040661}


Функцию-обертку реализуем в отдельном файле best_pokemon_service.py

Чтобы запустить API, нужно в терминале перейти в папку с кодом сервиса и ввести ```gunicorn --bind 0.0.0.0:8000 best_pokemon_service:app```  
Если ошибка ```Connection in use: ('0.0.0.0', 8000)```
- то либо делаем ```kill <номер, который у процесса, который занял порт>```
- либо пробуем вместо 8000 другой

После запуска API, можно им пользоваться 

In [94]:
features_test

,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation
586,Psychic,Flying,55,45,43,55,43,72,5
766,Rock,Dragon,58,89,77,45,45,48,6
722,Fire,NaN,59,59,58,90,70,73,6
580,Normal,Flying,80,115,80,65,55,93,5
542,Fire,Steel,91,90,106,130,106,77,4
...,...,...,...,...,...,...,...,...,...
525,Normal,NaN,85,80,70,135,75,90,4
338,Electric,NaN,70,75,60,105,60,105,3
606,Grass,Fairy,40,27,60,37,50,66,5
596,Water,Ground,75,65,55,65,55,69,5


In [96]:
header = {
    'Content-Type': 'application/json',
    'Accept': 'application/json'
}

resp = requests.post(
    "http://localhost:8000/predict", 
    data=json.dumps(features_test.to_json(orient='records')),
    headers=header
)

In [97]:
resp.status_code

200

In [98]:
print(
    f'Качество модели на тестовой выборке от модели в API:', 
    {roc_auc_score(target_test, pd.read_json(resp.json()['predictions']))}
)

Качество модели на тестовой выборке от модели в API: {0.989780387040661}


/tmp/ipykernel_425/2143769059.py:3: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  {roc_auc_score(target_test, pd.read_json(resp.json()['predictions']))}
